In [1]:
!apt install gcc g++ python3-dev build-essential -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.


In [2]:
!pip install signatory==1.2.6.1.9.0 iisignature esig pandas

In [3]:
import signatory
import torch
import iisignature
import numpy as np
import esig
import pandas as pd
import time
from typing import Callable, List, Dict
import json
from datetime import datetime
import platform
from IPython.display import display

def get_cpu_info():
    if platform.system() == "Linux":
        try:
            with open('/proc/cpuinfo', 'r') as f:
                for line in f:
                    if 'model name' in line:
                        return line.split(':')[1].strip()
        except:
            pass
    return platform.processor() or platform.machine()

def batch_esig(paths, depth):
    return np.array([esig.tosig.stream2sig(paths[i], depth) for i in range(len(paths))])[:,1:]

def signatory_sig(paths, depth, device):
    paths = torch.tensor(paths, device=device)
    return signatory.signature(paths, depth).to('cpu').numpy()

def time_function(func: Callable, number: int = 10) -> float:
    """Time a function over multiple runs and return average time in milliseconds"""
    times = []
    for _ in range(number):
        start = time.time()
        _ = func()
        times.append((time.time() - start) * 1000)  # Convert to milliseconds
    return np.mean(times)

def run_benchmark(batch_size: int, seq_len: int, n_features: int, depth: int) -> Dict:
    """Run benchmark for a specific configuration"""
    CPU_DEVICE = torch.device("cpu")
    GPU_DEVICE = torch.device("cuda")
    
    # Generate paths
    paths = np.random.randn(batch_size, seq_len, n_features)
    
    # Verify correctness
    iisig = iisignature.sig(paths, depth)
    signatorysig_cpu = signatory_sig(paths, depth, CPU_DEVICE)
    signatorysig_gpu = signatory_sig(paths, depth, GPU_DEVICE)
    esigsig = batch_esig(paths, depth)
    
    assert np.max(np.abs(iisig - signatorysig_cpu)) < 1e-8
    assert np.max(np.abs(iisig - signatorysig_gpu)) < 1e-8
    assert np.max(np.abs(iisig - esigsig)) < 1e-8
    
    # Time each implementation
    results = {
        'batch_size': batch_size,
        'seq_len': seq_len,
        'n_features': n_features,
        'depth': depth,
        'iisignature_time': time_function(lambda: iisignature.sig(paths, depth)),
        'esig_time': time_function(lambda: batch_esig(paths, depth)),
        'signatory_cpu_time': time_function(lambda: signatory_sig(paths, depth, CPU_DEVICE)),
        'signatory_gpu_time': time_function(lambda: signatory_sig(paths, depth, GPU_DEVICE))
    }
    
    return results

def run_parameter_sweep():
    """Run benchmarks varying one parameter at a time"""
    
    # Default parameters
    default_batch_size = 128
    default_seq_len = 100
    default_n_features = 3
    default_depth = 4
    
    # Parameter ranges
    batch_sizes = [32, 64, 128, 256, 512]
    seq_lens = [50, 100, 200, 500, 1000]
    depths = [2, 3, 4, 5, 6]
    
    results = []
    
    # Vary batch size
    for batch_size in batch_sizes:
        result = run_benchmark(
            batch_size=batch_size,
            seq_len=default_seq_len,
            n_features=default_n_features,
            depth=default_depth
        )
        results.append(result)
    
    # Vary sequence length
    for seq_len in seq_lens:
        result = run_benchmark(
            batch_size=default_batch_size,
            seq_len=seq_len,
            n_features=default_n_features,
            depth=default_depth
        )
        results.append(result)
    import platform
    # Vary depth
    for depth in depths:
        result = run_benchmark(
            batch_size=default_batch_size,
            seq_len=default_seq_len,
            n_features=default_n_features,
            depth=depth
        )
        results.append(result)
    
    # Convert to DataFrame
    df = pd.DataFrame(results)
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_filename = f'signature_benchmarks_{timestamp}.csv'
    df.to_csv(csv_filename, index=False)
    
    # Also save metadata about the system
    metadata = {
        'cpu_info': get_cpu_info(),
        'device_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
        'torch_version': torch.__version__,
        'signatory_version': signatory.__version__,
        'iisignature_version': iisignature.__version__,
        'esig_version': esig.get_version(),
        'timestamp': timestamp,
        'system': platform.system(),
        'python_version': platform.python_version(),
        'platform': platform.platform()
    }
    
    with open(f'signature_benchmarks_metadata_{timestamp}.json', 'w') as f:
        json.dump(metadata, f, indent=4)
    
    return df, metadata


print("Starting signature benchmarks...")
df, metadata = run_parameter_sweep()
print("\nBenchmarks complete. Results saved to CSV.")
print("\nSystem information:")
for key, value in metadata.items():
    print(f"{key}: {value}")

display(df)

Starting signature benchmarks...

Benchmarks complete. Results saved to CSV.

System information:
cpu_info: AMD Ryzen 9 5900X 12-Core Processor
device_name: NVIDIA GeForce RTX 4090
torch_version: 1.9.0
signatory_version: 1.2.6
iisignature_version: 0.24
esig_version: 0.9.8.3
timestamp: 20250105_132718
system: Linux
python_version: 3.7.10
platform: Linux-5.15.0-102-generic-x86_64-with-debian-buster-sid


,batch_size,seq_len,n_features,depth,iisignature_time,esig_time,signatory_cpu_time,signatory_gpu_time
0,32,100,3,4,2.416515,7.135034,1.091623,1.990914
1,64,100,3,4,4.843402,14.308977,1.315618,1.991820
2,128,100,3,4,9.657526,28.564882,1.512480,2.015328
3,256,100,3,4,19.282985,57.121038,2.099299,2.696323
4,512,100,3,4,38.657689,114.092994,3.095484,3.210521
5,128,50,3,4,4.774356,19.316483,1.000881,1.984787
6,128,100,3,4,9.653282,28.630018,1.697063,2.269053
7,128,200,3,4,19.441557,47.267127,3.092194,3.565240
8,128,500,3,4,48.693919,102.219582,7.181430,5.464315
9,128,1000,3,4,97.627950,194.751310,14.094281,8.780241


In [4]:
import torch
import torch.nn as nn
import signatory
import time
import numpy as np
import json
import pandas as pd
from datetime import datetime
import platform

def get_cpu_info():
    if platform.system() == "Linux":
        try:
            with open('/proc/cpuinfo', 'r') as f:
                for line in f:
                    if 'model name' in line:
                        return line.split(':')[1].strip()
        except:
            pass
    return platform.processor() or platform.machine()

class SigNet(nn.Module):
    def __init__(self, in_channels, out_dimension, sig_input_size, sig_depth):
        super(SigNet, self).__init__()
        self.dense1 = nn.Linear(in_channels, sig_input_size)
        self.signature = signatory.Signature(depth=sig_depth)
        sig_channels = signatory.signature_channels(channels=sig_input_size,
                                                    depth=sig_depth)
        self.linear = torch.nn.Linear(sig_channels, out_dimension)
        
    def forward(self, inp):
        dense_out = self.dense1(inp)
        y = self.signature(dense_out)
        z = self.linear(y)
        return z

def create_data(num_sample, seq_len, n_feature, n_ahead, device):
    X = torch.randn(num_sample, seq_len, n_feature, device=device)
    y = torch.randn(num_sample, n_ahead, device=device)
    return X, y

def train_model(model, X, y, batch_size, device, epochs=10):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()
    
    dataset = torch.utils.data.TensorDataset(X, y)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    losses = []
    training_time = 0
    
    for epoch in range(epochs):
        epoch_start = time.time()
        total_loss = 0
        
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        epoch_time = time.time() - epoch_start
        training_time += epoch_time
        avg_loss = total_loss/len(dataloader)
        losses.append(avg_loss)
    
    return {
        'final_loss': losses[-1],
        'training_time': training_time,
        'avg_epoch_time': training_time/epochs
    }

def run_benchmarks():
    # Parameters to test
    seq_lens = [100, 200, 350, 500]
    sig_input_sizes = [2, 4, 6, 10]
    depths = [2, 3, 4]
    
    # Fixed parameters
    batch_size = 128
    n_feature = 20
    n_ahead = 10
    epochs = 10
    
    results = []
    
    # Set up devices
    cpu_device = torch.device("cpu")
    gpu_device = torch.device("cuda")
    devices = [cpu_device, gpu_device]
    device_names = ["cpu", "gpu"]
    
    total_runs = len(seq_lens) * len(sig_input_sizes) * len(depths) * len(devices)
    current_run = 0
    
    for seq_len in seq_lens:
        num_sample = batch_size * 100 - 35  # Not exactly divisible by batch size
        
        for sig_input_size in sig_input_sizes:
            for depth in depths:
                for device, device_name in zip(devices, device_names):
                    current_run += 1
                    print(f"\nRun {current_run}/{total_runs}")
                    print(f"Parameters: seq_len={seq_len}, sig_input_size={sig_input_size}, depth={depth}, device={device_name}")
                    
                    # Create data
                    X, y = create_data(num_sample, seq_len, n_feature, n_ahead, device)
                    
                    # Create and train model
                    model = SigNet(n_feature, n_ahead, sig_input_size, depth)
                    t1 = time.time()
                    training_results = train_model(model, X, y, batch_size, device, epochs)
                    total_time = time.time() - t1
                    
                    results.append({
                        'seq_len': seq_len,
                        'sig_input_size': sig_input_size,
                        'depth': depth,
                        'device': device_name,
                        'total_time': total_time,
                        'training_time': training_results['training_time'],
                        'avg_epoch_time': training_results['avg_epoch_time'],
                        'final_loss': training_results['final_loss']
                    })
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    
    # Save results
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Save benchmark results
    csv_filename = f'model_benchmarks_{timestamp}.csv'
    df.to_csv(csv_filename, index=False)
    
    # Save metadata
    metadata = {
        'cpu_info': get_cpu_info(),
        'device_name': torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU',
        'torch_version': torch.__version__,
        'signatory_version': signatory.__version__,
        'timestamp': timestamp,
        'system': platform.system(),
        'python_version': platform.python_version(),
        'platform': platform.platform(),
        'batch_size': batch_size,
        'n_feature': n_feature,
        'n_ahead': n_ahead,
        'epochs': epochs
    }
    
    with open(f'model_benchmarks_metadata_{timestamp}.json', 'w') as f:
        json.dump(metadata, f, indent=4)
    
    return df, metadata


print("Starting model benchmarks...")
df, metadata = run_benchmarks()
print("\nBenchmarks complete. Results saved to CSV and JSON files.")
print("\nSystem information:")
for key, value in metadata.items():
    print(f"{key}: {value}")

Starting model benchmarks...

Run 1/96
Parameters: seq_len=100, sig_input_size=2, depth=2, device=cpu

Run 2/96
Parameters: seq_len=100, sig_input_size=2, depth=2, device=gpu

Run 3/96
Parameters: seq_len=100, sig_input_size=2, depth=3, device=cpu

Run 4/96
Parameters: seq_len=100, sig_input_size=2, depth=3, device=gpu

Run 5/96
Parameters: seq_len=100, sig_input_size=2, depth=4, device=cpu

Run 6/96
Parameters: seq_len=100, sig_input_size=2, depth=4, device=gpu

Run 7/96
Parameters: seq_len=100, sig_input_size=4, depth=2, device=cpu

Run 8/96
Parameters: seq_len=100, sig_input_size=4, depth=2, device=gpu

Run 9/96
Parameters: seq_len=100, sig_input_size=4, depth=3, device=cpu

Run 10/96
Parameters: seq_len=100, sig_input_size=4, depth=3, device=gpu

Run 11/96
Parameters: seq_len=100, sig_input_size=4, depth=4, device=cpu

Run 12/96
Parameters: seq_len=100, sig_input_size=4, depth=4, device=gpu

Run 13/96
Parameters: seq_len=100, sig_input_size=6, depth=2, device=cpu

Run 14/96
Paramet

In [5]:
df

,seq_len,sig_input_size,depth,device,total_time,training_time,avg_epoch_time,final_loss
0,100,2,2,cpu,6.856625,6.856008,0.685601,0.993986
1,100,2,2,gpu,5.945706,5.945136,0.594514,1.003136
2,100,2,3,cpu,8.727443,8.726954,0.872695,0.996604
3,100,2,3,gpu,7.273141,7.272586,0.727259,0.997808
4,100,2,4,cpu,10.959468,10.958965,1.095897,1.004278
...,...,...,...,...,...,...,...,...
91,500,10,2,gpu,10.473240,10.472604,1.047260,0.998275
92,500,10,3,cpu,141.727780,141.727265,14.172727,0.979513
93,500,10,3,gpu,16.732734,16.732118,1.673212,0.974684
94,500,10,4,cpu,1031.962024,1031.961468,103.196147,0.975251
